### Targeted information retrieval

We have seen how to parse a webpage, which retrieves the information without distinction.

But, in general, the purpose of scrapping is to automate the collection of targeted information on the web


In [ ]:
from bs4 import BeautifulSoup
import requests

Let's say I want to scrape the description of the latest movies released in theaters

So I go to the allociné website and try to find the tags that will give me links to the specific pages of these movies to get their summaries.

#### Recovery of the url of the pages of films newly released in the theaters

In [ ]:
url='http://www.allocine.fr/'
r = requests.get(url)
print(url, r.status_code)
soup = BeautifulSoup(r.content,'lxml')
soup

On your web browser (Chrome, Firefox,...), you can use the "inspect" function (right click -> inspect) and drag your mouse to the areas of the page that interest you. At the same time I will see the html script move to the instructions of the html script in question. 

That's how you find the tags that you are interested in.

I notice that the relative link of the web page specific to the new movie on the poster is stored in these tags:

```html
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=235582.html" title="Le Grand Bain">Le Grand Bain</a>
```


In [ ]:
for p in soup.find_all('a'):
    print (p.text)

This time, the site is more difficult to "extract". Let's use much more specific parameters to the search function `find_all`.

In [ ]:
# In addition to the tag a, which is easily identifiable, we notice some additional 
# information such as the value of the class variable of these identical tags.
for elem in soup.find_all('a',attrs={"class" :"meta-title meta-title-link"}):
    print(elem)

#### Recovery of `href`

We have noticed the presence of `href` links to the pages that interest us. Let's go get them back:

In [ ]:
for elem in soup.find_all('a',attrs={"class" :"meta-title meta-title-link"}):
    print(elem.get('href'))
    # return a list

Can you retrieve the titles for me via the search for "title" in the items of the previous list?

#### Get summary

Let's start by building the url that we will use to retrieve the summaries

Start by putting the `href` values in a list of links


In [ ]:
links=[]
for elem in soup.find_all('a',attrs={"class" :"meta-title meta-title-link"}):
    # I simply put all of thisin a list
    links.append(elem.get('href'))
links

The absolute url of the searched movie pages is built in this form: http://www.allocine.fr/film/fichefilm_gen_cfilm=243835.html

It is therefore necessary to repeat the previous list and build the absolute urls for our search

It's up to you to play.

NB: Do not take the links for the shows(series)

In [ ]:
links_movie=['http://www.allocine.fr'+ elem for elem in links if 'film' in elem]
links_movie

Finally, on each page, the title and synopsis must be retrieved. Let's try for a movie, the first of the list

In [ ]:
url=links_movie[0]
r = requests.get(url)
print(url, r.status_code)
soup = BeautifulSoup(r.content,'lxml')
soup

For title 
```html
<div class="titlebar-title titlebar-title-lg">Le Grand Bain</div>
```
For the synopsis

```html
<div class="content-txt " itemprop="description"

 
              C’est dans les couloirs de leur piscine municipale que Bertrand, Marcus, Simon, Laurent, Thierry et les autres s’entraînent sous l’autorité toute relative de Delphine, ancienne gloire des bassins. Ensemble, ils se sentent libres et utiles. Ils vont mettre toute leur énergie dans une discipline jusque-là propriété de la gent féminine : la natation synchronisée. Alors, oui c’est une idée plutôt bizarre, mais ce défi leur permettra de trouver un sens à leur vie...
    
      </div>
```

In [ ]:
for elem in soup.find_all('div',attrs={"class" :"titlebar-title titlebar-title-lg"}):
    # Just like that
    print(elem.text)
    
for elem in soup.find_all('div',attrs={"class" :"content-txt "}):
    # Just like that
    print(elem.text)

1) Automate this script for the entire list

2) Put the information in three lists (film_links, title, synopsis)

3) Create a dataframe that includes these three informations in three associated columns

4) Save this dataframe in a csv file

And here's your first real scrap, you're real hackers now.

In [ ]:
import time
import random
from random import randint

title=[]
synopsis=[]

for link in links_movie:
    
    url=link
    # I slow down the frequency of requests to avoid being identified and therefore ban from the site
    time.sleep(random.uniform(1.0, 2.0))
    r = requests.get(url)
    print(url, r.status_code)
    soup = BeautifulSoup(r.content,'lxml')
    
    
    for elem in soup.find_all('div',attrs={"class" :"titlebar-title titlebar-title-lg"}):
        title.append(elem.text.strip())

    for elem in soup.find_all('div',attrs={"class" :"content-txt "}):
        synopsis.append(elem.text.strip())
        
# I check the length of the lists before creating the df
len(title),len(synopsis),len(links_movie)

In [ ]:
import pandas as pd
df=pd.DataFrame({'Titre':titre})
df['synopsis']=synopsis
df['liens']=liens_film

In [ ]:
df.to_csv('./assets/allo_cine.csv', index=False)